# Label Scenarios based on Land-cover Variables

In [1]:
import pyam
import functools
import operator
import pathlib

import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

# Read in only minimal data for all regional datasets

In [2]:
data_path = pathlib.Path('../data')

In [3]:
# we use this to catch all scenarios with a temperature outcome
temp_variable = 'AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile'
variables_of_interest = [
    # the rest we will check how many models report them
    'Land Cover|Forest',
    'Land Cover|Pasture',
    'Land Cover|Cropland',
]
all_vars = [temp_variable] + variables_of_interest

In [4]:
all_files = [
    'AR6_Scenarios_Database_World_v1.0.csv',
    'AR6_Scenarios_Database_R5_regions_v1.0.csv',
    'AR6_Scenarios_Database_R10_regions_v1.0.csv',
]

In [5]:
dfs = []
for fname in all_files:
    print(fname)
    df = pd.read_csv(data_path / fname)
    df.columns = df.columns.str.lower()
    dfs.append(df[df.variable.isin(all_vars)])

AR6_Scenarios_Database_World_v1.0.csv
AR6_Scenarios_Database_R5_regions_v1.0.csv
AR6_Scenarios_Database_R10_regions_v1.0.csv


In [6]:
df = pyam.concat([pyam.IamDataFrame(df) for df in dfs])
df.load_meta(data_path / 'AR6_Scenarios_Database_metadata_indicators_v1.0.xlsx')

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - WARNING: Reading meta indicators for 1716 out of 1717 scenarios, ignoring 588 scenarios from file


# Code each scenario with whether they have variables of interest

In [7]:
region_list = [
    ('World', 'Global '),
    ('R5ASIA', 'R5 '),
    ('R10AFRICA', 'R10 ')
]

region_variables = []
for variable in all_vars:
    for region, prepend in region_list:
        name = prepend + variable
        region_variables.append(name)
        df.set_meta_from_data(
            name=name, 
            method=lambda x: True,
            variable=variable, 
            year=2100,
            region=region,
        )

pyam.core - WARNING: Filtered IamDataFrame is empty!
pyam.core - WARNING: Filtered IamDataFrame is empty!


In [8]:
for _, prepend in region_list: # do they have all variables we want together
    name = prepend + 'all'
    df.meta[name] = functools.reduce(operator.and_, [df.meta[prepend + v].fillna(False) for v in variables_of_interest])
    df.meta[name][~df.meta[name]] = np.nan

C:\Users\gidden\AppData\Local\Temp\ipykernel_16752\4230539601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.meta[name][~df.meta[name]] = np.nan
C:\Users\gidden\AppData\Local\Temp\ipykernel_16752\4230539601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.meta[name][~df.meta[name]] = np.nan
C:\Users\gidden\AppData\Local\Temp\ipykernel_16752\4230539601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.meta[name][~df.m

In [9]:
metadata_of_interest = region_variables + [prepend + 'all' for _, prepend in region_list]
data = df.meta.groupby('Category')[metadata_of_interest].count()
data.T.sort_values(by="C1",ascending=False).round(decimals = 0) 

Category,C1,C2,C3,C4,C5,C6,C7,C8
Global AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile,97,133,311,159,212,97,164,29
Global Land Cover|Forest,75,107,238,140,188,81,100,9
R5 Land Cover|Forest,74,107,238,140,188,81,99,9
Global Land Cover|Pasture,72,106,232,138,186,81,99,9
Global Land Cover|Cropland,72,107,232,138,186,81,99,9
Global all,72,106,232,138,186,81,99,9
R5 Land Cover|Pasture,71,106,232,138,186,81,98,9
R5 Land Cover|Cropland,71,107,232,138,186,81,98,9
R5 all,71,106,232,138,186,81,98,9
R10 Land Cover|Forest,57,84,174,90,139,54,49,5


In [34]:
data = df.meta.groupby('Category')[metadata_of_interest].count()
data.T.sort_values(by="C1",ascending=False).round(decimals = 0) 
#data = data.T / data['Global AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile'] * 100
#data.sort_values(by="C1",ascending=False).round(decimals = 0) 

Category,C1,C2,C3,C4,C5,C6,C7,C8
Global AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile,97,133,311,159,212,97,164,29
Global Land Cover|Forest,75,107,238,140,188,81,100,9
R5 Land Cover|Forest,74,107,238,140,188,81,99,9
Global Land Cover|Pasture,72,106,232,138,186,81,99,9
Global Land Cover|Cropland,72,107,232,138,186,81,99,9
Global all,72,106,232,138,186,81,99,9
R5 Land Cover|Pasture,71,106,232,138,186,81,98,9
R5 Land Cover|Cropland,71,107,232,138,186,81,98,9
R5 all,71,106,232,138,186,81,98,9
R10 Land Cover|Forest,57,84,174,90,139,54,49,5


In [24]:
idx = (
    ['Global AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile'] + 
    [x for x in data.index if 'R5' in x][1:]
)
idx

['Global AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile',
 'R5 Land Cover|Forest',
 'R5 Land Cover|Pasture',
 'R5 Land Cover|Cropland',
 'R5 all']

In [29]:
table = (
    data
    .loc[idx]
    .rename(index={'Global AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile': 
                  'Surface Temperature'})
    .sort_values(by="C1",ascending=False)
    .round(decimals = 0)
    [['C1', 'C2', 'C3']]
)
table

Category,C1,C2,C3
Surface Temperature,100.0,100.0,100.0
R5 Land Cover|Forest,76.0,80.0,77.0
R5 Land Cover|Pasture,73.0,80.0,75.0
R5 Land Cover|Cropland,73.0,80.0,75.0
R5 all,73.0,80.0,75.0


In [40]:
len(df.filter(**{'R5 all': True}).index)

1216

In [44]:
df.meta['Vetting_historical']

model              scenario              
AIM/CGE 2.0        ADVANCE_2020_1.5C-2100    Fail
                   ADVANCE_2020_Med2C        Fail
                   ADVANCE_2020_WB2C         Fail
                   ADVANCE_2030_Med2C        Fail
                   ADVANCE_2030_Price1.5C    Fail
                                             ... 
WITCH-GLOBIOM 4.4  CD-LINKS_NPi2020_1000     Pass
                   CD-LINKS_NPi2020_1600     Pass
                   CD-LINKS_NPi2020_400      Pass
                   CD-LINKS_NoPolicy         Pass
POLES GECO2019     NDCMCS                     NaN
Name: Vetting_historical, Length: 1717, dtype: object

In [49]:
len(df.filter(Category=['C3', 'C5', 'C6', 'C7', 'C4', 'C1', 'C2', 'C8']).index)

1202

In [32]:
table.to_excel('~/Downloads/table.xlsx', index=True)

# Add cumulative CDR

In [11]:
idf = pd.read_csv(data_path / 'AR6_Scenarios_Database_World_v1.0.csv')
idf = pyam.IamDataFrame(idf[idf.Variable.isin(['Carbon Sequestration|CCS|Biomass'])])
idf = idf.interpolate(range(2010, 2101))
idf = idf.filter(year=range(1990, 2015), keep=False)
idf.set_meta_from_data(
    name='Cumulative BECCS', 
    method=np.sum,
    variable='Carbon Sequestration|CCS|Biomass', 
    region='World',
)
df.meta['Cumulative BECCS'] = idf.meta['Cumulative BECCS']

# Export

In [12]:
df.meta.to_excel(
    data_path / 'AR6_Scenarios_Database_metadata_indicators_v1.0_variable_coverage.xlsx',
    sheet_name='meta',
)

# Basic Stats

In [13]:
meta = pd.read_excel(data_path / 'AR6_Scenarios_Database_metadata_indicators_v1.0.xlsx', sheet_name='meta')

In [14]:
len(meta)

2304

In [15]:
meta.groupby('Category')['Model'].count()

Category
C1     97
C2    133
C3    311
C4    159
C5    212
C6     97
C7    164
C8     29
Name: Model, dtype: int64

In [16]:
meta.groupby('Category')['Model'].count().sum()

1202